### 자연어 감정분석
- 감성사전 기반 : 미리정의된 감성 단어 사전 사용(규칙 기반)
    - TextBlob, AFINN, VADER
- 머신러닝 기반 : 데이터로부터 패턴 학습(통계 기반)
    - TF-IDF 벡터화
    - 선형회귀
    - 로지스틱회귀
    - F1 Score, Recision, Recall -> classification report

### 사용 데이터
- NLTK 영화 리뷰(2000개)
- 다음영화리뷰
    

### 알고리즘
- TextBlob  사전기반 감성분석
- AFINN     감정 점수 매핑
- VADER(Valance Aware Dictionary) 소셜미디어 최적화 감성분석
- TF-IDF    텍스트 벡터화
- Multinomial Naive Bayes 확률 기반 분류

In [89]:
# TextBlob
# 이 영화는 정말 좋고 재미있다
    # 좋다 +1(긍정)
    # 재미있다 +1(긍정)
    # +2 > 0 --> 긍정(pos) 분류
# Polarity(극성도)  (긍정단어수 개수 - 부정단어 개수) / 전체 단어 개수
# -1.0 ~ +1.0
# 0 중립
# Subjectivity(주관성)
# 0.0 ~ 1.0
# 0 : 객관적  1 : 주관적

# 문맥 무시하고 단어 극성만 고려
# 이 영화는 나쁘지 않다 -> 나쁘다(-) 않다(-) 로 인식
# 장점 : 빠른 속도, 학습 불필요
# 사용 : 실시간 감성분석, 스트리밍데이터

In [90]:
# %conda install textblob

In [91]:
from textblob import TextBlob, Word
# import nltk
# nltk.download('brown')

text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob = TextBlob(text)
print(blob.sentences)
print(blob.words)
print(blob.tags)
print(blob.noun_phrases)
print('-'*100)
# 감성분석
print(blob.sentiment)
print(f'polarity: {blob.sentiment.polarity}')
print(f'subjectivity: {blob.sentiment.subjectivity}')

[Sentence("TextBlob is amazingly simple to use."), Sentence("What a wonderful library for NLP!")]
['TextBlob', 'is', 'amazingly', 'simple', 'to', 'use', 'What', 'a', 'wonderful', 'library', 'for', 'NLP']
[('TextBlob', 'NNP'), ('is', 'VBZ'), ('amazingly', 'RB'), ('simple', 'JJ'), ('to', 'TO'), ('use', 'VB'), ('What', 'WP'), ('a', 'DT'), ('wonderful', 'JJ'), ('library', 'NN'), ('for', 'IN'), ('NLP', 'NNP')]
['textblob', 'wonderful library', 'nlp']
----------------------------------------------------------------------------------------------------
Sentiment(polarity=0.5, subjectivity=0.6785714285714286)
polarity: 0.5
subjectivity: 0.6785714285714286


In [92]:
# AFINN(Lexicon-Based)  감성사전
# 각 단어의 -5 ~ +5의 점수를 부여하고 합산
# 이 영화는 좋지만 좋지 않은 부분도 있다
    # 좋다 +3 좋다 +3 나쁘다 -3 = +3 > 0 긍정
# score = sum(word_sentiment_value)
# 분류규칙 score > 0 긍정   score < 0 부정
# 이모티콘지원
# 강도표현 인식 very, really 등

# 강조 수정자(intensifiers)
    # 매우좋다 = 1.5 * (좋다의 점수)

# AFINN vs TextBlob
# AFINN : 더 정확한 점수 매핑
# TextBlob : 더 일반적인 접근

In [93]:
# %pip install afinn

In [94]:
from afinn import Afinn
af = Afinn()
text1 = "TextBlob is amazingly simple to use."
text2 = "What a wonderful library for NLP!"
score1 = af.score(text1)
score2 = af.score(text2)
score1, score2

(0.0, 4.0)

In [95]:
# VADER 소셜미디어 텍스트에 최적화
# 이 영화는 정말정말 훌륭해!!!
# 훌륭하다 (기본) + 0.7  정말정말 (강조) * 1.5 
# !!! (문장부호강조) * 1.2
# 4개의 감정 지수
    # positive 긍정 확률 0 ~ 1
    # negative 부정 확률
    # neutral 중립 확률
    # compound 종합점수 -1 ~ 1
# score = compound_score / sqrt(compound_score**2 + 0.0625)
# score >= 0.05 긍정
# score <= -0.05 부정
# 그 사이는 중립
# 대소문자 구분 AMAZING amazing 다른 점수 부여
# :) 긍정   :( 부정

In [96]:
# import nltk
# nltk.download('vader_lexicon')

In [97]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyer = SentimentIntensityAnalyzer()
sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]
for s in sentences:
    scores = analyer.polarity_scores(s)
    print(f'문장 : {s}')
    print(f'점수 : {scores}')

문장 : I love this product! It's absolutely amazing 😍
점수 : {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
문장 : This is the worst movie I've ever seen...
점수 : {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
문장 : The food was okay, not great but not bad either.
점수 : {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
문장 : I’m REALLY happy with the results!!!
점수 : {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
문장 : Not good at all. I’m disappointed.
점수 : {'neg': 0.579, 'neu': 0.421, 'pos': 0.0, 'compound': -0.6711}


In [98]:
import nltk
from nltk.corpus import movie_reviews
from textblob import TextBlob
from afinn import Afinn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import pandas as pd

# nltk 데이터 다운로드
nltk.download('movie_reviews', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# 영화 리뷰 데이터 로드
fileids = movie_reviews.fileids()

In [99]:
reviews = [movie_reviews.raw(fileid) for fileid in fileids[:50] ] + \
    [movie_reviews.raw(fileid) for fileid in fileids[-50:] ]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids[:50] ] + \
    [movie_reviews.categories(fileid)[0] for fileid in fileids[-50:] ]
len(reviews), categories.count('pos'), categories.count('neg')

(100, 50, 50)

In [100]:
# 1. TextBlob
def sentiment_textblob(docs):
    return ['pos' if TextBlob(doc).sentiment.polarity > 0 else 'neg' for doc in docs ]
predictions_textblob = sentiment_textblob(reviews)
accuracy_textblob = accuracy_score(categories, predictions_textblob)
print(f'정확도 : {accuracy_textblob}')

정확도 : 0.61


In [101]:
# 2. AFINN 
def sentiment_afinn(docs):
    afn = Afinn(emoticons=True)
    return [ 'pos' if afn.score(doc) > 0 else 'neg' for doc in docs ]
predictions = sentiment_afinn(reviews)
accuracy = accuracy_score(categories, predictions)
print(f'정확도 : {accuracy:.1f}')

정확도 : 0.7


In [102]:
# 3. VADER 
def sentiment_vader(docs):
    analyzer = SentimentIntensityAnalyzer()
    return [ 'pos' if analyzer.polarity_scores(doc)['compound'] > 0 else 'neg' for doc in docs ]
predictions = sentiment_vader(reviews)
accuracy = accuracy_score(categories, predictions)
print(f'정확도 : {accuracy:.1f}')

정확도 : 0.6


In [103]:
# 머신러닝 기반 감성분석
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [104]:
# 나이브 베이즈

# 베이즈 정리
# "좋다" 단어를 본후 이 리뷰가 긍정일 확률
# p(긍정 | '좋다') = p('좋다' | 긍정) * p(긍정) / p('좋다')

In [105]:
# 데이터 분할
dataset = train_test_split(
    reviews, categories, test_size=0.2, random_state=42, stratify=categories)

In [108]:
# tf-idf 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
x_train = vectorizer.fit_transform(dataset[0])
x_test = vectorizer.transform(dataset[1])
y_train = dataset[2]
y_test = dataset[3]

In [ ]:
# 1. mnb
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train, y_train)
predict = mnb_clf.predict(x_test)
print( classification_report(y_test, predict) )

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [110]:
# LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
predict = lr.predict(x_test)
print( classification_report(y_test, predict) ) 


              precision    recall  f1-score   support

         neg       0.57      0.80      0.67        10
         pos       0.67      0.40      0.50        10

    accuracy                           0.60        20
   macro avg       0.62      0.60      0.58        20
weighted avg       0.62      0.60      0.58        20



In [111]:
# 성능향상
# 소문자변환 - 연속된 문자열중에 3글자 이상 - 어간추출(형태소분석) - 불용어 제거
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [ ]:
def custom_tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'[\w]{3,}')
    tokens = tokenizer.tokenize(text)
    porter = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    return [porter.stem(token) for token in tokens if token not in stop_words]
vector = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    max_features=1000,
    min_df=5,
    max_df=0.5,
    token_pattern=r'[\w]{3,}'
)
x_train = vector.fit_transform(dataset[0])
x_test = vector.transform(dataset[1])

c:\Users\playdata2\miniconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [116]:
def evaluate_model(model):
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    print( classification_report(y_test, predict) )

In [ ]:
evaluate_model(LogisticRegression())

              precision    recall  f1-score   support

         neg       0.80      0.80      0.80        10
         pos       0.80      0.80      0.80        10

    accuracy                           0.80        20
   macro avg       0.80      0.80      0.80        20
weighted avg       0.80      0.80      0.80        20



In [118]:
evaluate_model(MultinomialNB())

              precision    recall  f1-score   support

         neg       0.75      0.60      0.67        10
         pos       0.67      0.80      0.73        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20

